# OpenStreetMap Data Wrangling with MongoDB
## Data Wrangling Project
#### Data Analyst Nanodegree (Udacity)
Project submission by Edward Minnett (ed@methodic.io).

July 26th 2016. (Revision 1)

----------

## Overview

This project seeks to apply data munging techniques to first analyse and clean Open Street Map data for a single city and then perform an exploratory anlaysis of the data after importing it into a MongoDB collection.

I chose to explore the Open Street Map data for Oxfor England as for the last couple of years I have lived in a small village south of Oxford. Having visited the city on quite a few occasions, I am confident that there is a lot of intersting information to be dsicovered through this analysis.

The data for this project was aquired from Map Zen. The compressed Oxford, England OSM XML data set can be downloaded by following this [link](https://s3.amazonaws.com/metro-extracts.mapzen.com/oxford_england.osm.bz2). The OSM file is 5.0 MB compressed and 66.2 MB uncompressed.

In [1]:
# The following is needed at the biging of the notebook to make sure the cells execute ok.
import sys
sys.path.append('./case_study_files')

data_name = "oxford_england"
OSMFILE = "{}.osm".format(data_name)

## Problems encountered in the map

As you would expect from any crowd-sourced data, there is evidence of inconsistent naming conventions as well as evidence of human error when the data was eneterd. The two main areas of inconsistency that affect this project is in the names used to describe the type of record in the data and the strings used as values within the data. It isn't feasible to analyise all values in the data, but street names are one type of value where inconsistency in values has a significant impact on the quality of the data. 

The following analysis attempts to find problems with tag key names along with suggested fixes for the problems. There is a similar analysis and set of recommendations for fixing the values used for street names.

### Tag key name problems

In [2]:
# Find problems with tag names
import tags as tags_processor
tag_problems = tags_processor.process_map(OSMFILE)

# Additional key categories have been added in an attempt to catch additional key name patterns and minimise 
# the number of keys that fall within the 'other' category.

# The problems are defined as follows:
#   "lower", for tags that contain only lowercase letters and are valid,
#   "upper", for tags that contain upercase letters,
#   "lower_colon", for otherwise valid tags with a colon in their names,
#   "uper_colon", for tags contain both uppercase strings delimited by a colon,
#   "multiple_colons", for tags contain more than two colon seperated strings,
#   "numbers", for tags that contain a digit, and
#   "problemchars", for tags with problematic characters, and
#   "other", for other tags that do not fall into the other three categories.
print "The number of keys in each of the 'problem' categories:"
print tag_problems['counts']
unique_key_names = tags_processor.unique_tag_keys(OSMFILE)
print "There are {} unique tag key names in the data set.".format(len(unique_key_names))

The number of keys in each of the 'problem' categories:
{'problemchars': 5, 'upper': 120, 'lower': 143057, 'upper_colon': 7638, 'numbers': 769, 'multiple_colons': 608, 'lower_colon': 47850, 'other': 1}
There are 1011 unique tag key names in the data set.


It is the tags that fall under 'problemchars' that will be particularly problematic as these tag key names aren't valid key names in MongoDB. Let's see what these tags look like and how they could be fixed.

In [3]:
tag_problems['keys']['problemchars']

['leaving for now',
 'fee:amount:box_van&minibus',
 'note:0.1',
 'note:0.2',
 'note 2']

These issues can be easily fixed by making the following replacements:

- Replace ' ' with '\_'
- Replace '&' with '\_and\_'
- Replace '.' with '\_'

The logic to do this has been added to the case_study_files/data.py so that the key values are tidied when the elements are being shaped.

We will also add logic to handle the keys that fall under 'upper' and 'upper_colon'. This can be fixed by ensuring that shape_element uses the lowercase version of the strings.

The key names that fall under 'multiple_colons' can be handled by adding additional nesting within the JSON data created by shape_element.

Tag key names that fall within the 'numbers' category are useful to know about, but numbers are valid characters in MongoDB strings so no additional processing is required.

It is also worth looking at the one remaining key that fall under 'other' to see why it doesn't fall withing one of the other categories.

In [4]:
set(tag_problems['keys']['other'])

{'name:sr-Latn'}

Like the tag key names that contain numbers, the hyphen is also a valid character in a MongoDB key ao this one one 'other' key name is fine as it is.

### Street name problems

In [161]:
# Find problems with street names
import audit as street_name_auditor
street_name_auditor.audit(OSMFILE)

defaultdict(set,
            {'1': {'Avenue 1'},
             '2': {'Avenue 2'},
             '3': {'Avenue 3'},
             '4': {'Avenue 4'},
             "Aldate's": {"St Aldate's"},
             'Ave': {'Waverly Ave'},
             'Barr': {'Upper Barr'},
             'Bridge': {'Folly Bridge'},
             'Broadway': {'The Broadway'},
             'Buildings': {'Manor Buildings'},
             'Castle': {'Oxford Castle'},
             'Centre': {'Fairfax Centre'},
             'Centremead': {'Centremead'},
             'Chorefields': {'Chorefields'},
             'Clements': {'St Clements'},
             'Cloisters': {'Temple Cloisters'},
             'Close': {'Acland Close',
              'Acre Close',
              'Amory Close',
              'Ashcroft Close',
              'Ashroft Close',
              'Barn Close',
              'Bartlemas Close',
              'Benouville Close',
              'Blackburn Close',
              'Broad Close',
              'Browns Close',

The large majority of these street names are valid, but there are a few problems. There are a few cases of abbreviated street names such as 'Ave', 'Rd' and 'St'. There are also several cases of punctuations mistakes such as 'Way?' and 'Way,'. There are cases of both 'road' and 'way' being lowercase. There is also a typo where 'Reliuance Way' should be 'Reliance Way'. The oddest problems are the cases of 'Avenue 1' through 'Avenue 4'. In Kennington just outside of Oxford the main street is called 'The Avenue'. I believe these street names may be mistakes or parsing issues. They don't match any obvious street names and as a result will be treated as anomolies and ignored.

Logic has been added to case_study_files/data.py to clean up these inconsistencies.

## Load the Data

In [6]:
# This snippet uses the process_map function from the case study data.py script. This process is idempotent and will 
# only load more data into MongoDB if any of the following happens:
#     - The number of records in the MongoDB collection matching the data_name variable has changed.
#     - The value of the data_name variable changes.
#     - The number of records in the OSM file matching data_name changes.

import data as data_processor
data = data_processor.process_map(OSMFILE, True)

from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017")
db = client.oxford_england_sample
collection = getattr(db, data_name)

collection_count = collection.count()

# If the collection size dowsn't match the data size, load/reload the data.
if collection_count != len(data):
    if collection_count > 0:
        collection.drop()
    collection.insert_many(data)

collection_count = collection.count()
sample_record = collection.find_one()
print "Number of records in the {} MongoDB collection: {}".format(data_name, collection_count)
print "A sample record from the {} MongoDB collection: {}".format(data_name, sample_record)

Number of records in the oxford_england MongoDB collection: 321322
A sample record from the oxford_england MongoDB collection: {u'created': {u'changeset': u'10706805', u'version': u'4', u'uid': u'27408', u'timestamp': u'2012-02-16T23:08:31Z', u'user': u'Andrew Chadwick'}, u'pos': [51.6994959, -1.2645627], u'visible': None, u'_id': ObjectId('57963746d0958625140ac407'), u'type': u'node', u'id': u'194502'}


## Overview of the data

In [7]:
# Size of the OSM and JSON files.
import os
def get_size_in_mb_of_relative_file(file_name):
    wd = %pwd
    return os.stat(wd + '/' + file_name).st_size / 1000.0 / 1000.0

for file_name in [OSMFILE, OSMFILE + '.json']:
    file_size = get_size_in_mb_of_relative_file(file_name)
    print "{} is {} MB in size.".format(file_name, file_size)

oxford_england.osm is 66.214418 MB in size.
oxford_england.osm.json is 99.004354 MB in size.


In [8]:
# Number of records:
print "Number of records in the {} MongoDB collection: {}".format(data_name, collection_count)

Number of records in the oxford_england MongoDB collection: 321322


In [9]:
# Number of nodes:
num_nodes = collection.find({"type":"node"}).count()
print "Number of 'node' records in the {} MongoDB collection: {}".format(data_name, num_nodes)

Number of 'node' records in the oxford_england MongoDB collection: 274664


In [10]:
# Number of ways:
num_ways = collection.find({"type":"way"}).count()
print "Number of 'way' records in the {} MongoDB collection: {}".format(data_name, num_ways)

Number of 'way' records in the oxford_england MongoDB collection: 46600


In [11]:
# Number of unique users:
num_unique_users = len(collection.distinct("created.user"))
print "Number of unique 'users' in the {} MongoDB collection: {}".format(data_name, num_unique_users)

Number of unique 'users' in the oxford_england MongoDB collection: 567


In [15]:
# Number of unique amenity types:
num_unique_amenities = len(collection.distinct("amenity"))
print "Number of unique 'amenity' values in the {} MongoDB collection: {}".format(data_name, num_unique_amenities)

Number of unique 'amenities' in the oxford_england MongoDB collection: 127


In [16]:
# Number of unique cuisine types:
num_unique_cuisines = len(collection.distinct("cuisine"))
print "Number of unique 'cuisine' values in the {} MongoDB collection: {}".format(data_name, num_unique_cuisines)

Number of unique 'cuisine' values in the oxford_england MongoDB collection: 57


In [105]:
# Number of universities and colleges:
num_colleges = collection.find({"amenity": {"$in": ["university", "college"]}}).count()
print "Number of 'university' and 'college' records in the {} MongoDB collection: {}".format(data_name, num_colleges)

Number of 'university' and 'college' records in the oxford_england MongoDB collection: 119


## Other ideas about the datasets

Oxford is a very intersting city not simply because of the history and prestige of Oxford University, but also because of its density and diversity of amenities woven within a fabric of well integrated network of pedestrian, bicycle, and public transport routes. It would be very interesting to explore the relationships between these amenities and the various networks that allow people to move around the city.

The outcome of analysis could be very helpful in planning activities and events within the city as well as providing information about which parts of the city are likely to be busiest during the tourist season.

That said, the OSM data for Oxford may not make such an analysis very easy to undertake. For example, data about the bus and rail networks are encoded using the NAPTAN schema ([NAPTAN stands for National Public Transport Access Nodes](https://www.gov.uk/government/publications/national-public-transport-access-node-schema)) but this information does not always contain information about street names.

In [143]:
print "A bus stop node without street information."
print collection.find_one({"naptan": {"$exists": 1}, "highway": "bus_stop"})
print "\n"
print "A bus stop node with street information."
print collection.find_one({"naptan.Street": {"$exists": 1}, "highway": "bus_stop"})

A bus stop node without street information.
{u'direction': u'W', u'name': u'Herschel Crescent', u'journeys': u'6', u'created': {u'changeset': u'8007524', u'version': u'6', u'uid': u'74570', u'timestamp': u'2011-04-29T22:38:18Z', u'user': u'Richard Mann'}, u'pos': [51.7249536, -1.2152904], u'visible': None, u'frequency': u'0', u'naptan': {u'Bearing': u'W'}, u'source': u'photograph', u'bus_routes': u'16A', u'_id': ObjectId('57963746d0958625140add4d'), u'type': u'node', u'id': u'16640101', u'highway': u'bus_stop'}


A bus stop node with street information.
{u'direction': u'W', u'name': u'Long Lane', u'journeys': u'6', u'created': {u'changeset': u'8007524', u'version': u'6', u'uid': u'74570', u'timestamp': u'2011-04-29T22:38:18Z', u'user': u'Richard Mann'}, u'pos': [51.7249171, -1.2181715], u'note': u'NAPTAN node in wrong place, and erroneously marked CUS', u'visible': None, u'frequency': u'0', u'naptan': {u'Bearing': u'W', u'Indicator': u'Opp Sheldon Way', u'BusStopType': u'CUS', u'Common

### The top 10 most common amenities

In [144]:
results = collection.aggregate([
        {"$match": {"amenity": {"$exists": 1}}}, 
        {"$group": {"_id": "$amenity", "count": {"$sum": 1}}}, 
        {"$sort": {"count": -1}}, 
        {"$limit": 10}
    ])
for result in results:
    print result

{u'count': 741, u'_id': u'parking'}
{u'count': 521, u'_id': u'bicycle_parking'}
{u'count': 332, u'_id': u'post_box'}
{u'count': 187, u'_id': u'bench'}
{u'count': 164, u'_id': u'pub'}
{u'count': 163, u'_id': u'place_of_worship'}
{u'count': 142, u'_id': u'telephone'}
{u'count': 130, u'_id': u'restaurant'}
{u'count': 110, u'_id': u'cafe'}
{u'count': 102, u'_id': u'school'}


### The top 10 most common types of cuisine

In [25]:
results = collection.aggregate([
        {"$match": {"cuisine": {"$exists": 1}}}, 
        {"$group": {"_id": "$cuisine", "count": {"$sum": 1}}}, 
        {"$sort": {"count": -1}}, 
        {"$limit": 10}
    ])
for result in results:
    print result

{u'count': 31, u'_id': u'chinese'}
{u'count': 26, u'_id': u'sandwich'}
{u'count': 20, u'_id': u'indian'}
{u'count': 15, u'_id': u'coffee_shop'}
{u'count': 11, u'_id': u'fish_and_chips'}
{u'count': 11, u'_id': u'burger'}
{u'count': 11, u'_id': u'pizza'}
{u'count': 10, u'_id': u'italian'}
{u'count': 6, u'_id': u'thai'}
{u'count': 5, u'_id': u'asian'}


### Top 10 most commonly found places to eat

In [145]:
results = collection.aggregate([
        {"$match": {"amenity": {"$in": ["restaurant", "cafe", "pub", "fast_food", "delicatessen"]}}}, 
        {"$match": {"name": {"$exists": 1}}},
        {"$group": {
                "_id": "$name", 
                "amenity": {"$first": "$amenity"}, 
                "cuisine": {"$push": "$cuisine"}, 
                "count": {"$sum": 1}
            }},
        {"$project": {
                "_id": 0, 
                "count": 1, 
                "name": "$_id", 
                "type" : {"$concat": [
                        "$amenity", " - ", 
                        {"$ifNull": [{"$arrayElemAt": ["$cuisine", 0 ]}, "unknown cuisine"] }
                    ]}
            }},
        {"$sort": {"count": -1}}, 
        {"$limit": 10}
    ])

for result in results:
    print "Count {}: {} ({})".format(result['count'], result['name'], result['type'])

Count 6: The Red Lion (pub - unknown cuisine)
Count 5: Taylors (restaurant - sandwich)
Count 5: La Croissanterie (cafe - unknown cuisine)
Count 4: The White Hart (pub - unknown cuisine)
Count 3: Mission Burrito (fast_food - mexican)
Count 3: McDonald's (fast_food - burger)
Count 3: Subway (fast_food - sandwich)
Count 3: Pizza Hut (restaurant - pizza)
Count 3: Mortons (cafe - sandwich)
Count 3: Costa Coffee (cafe - unknown cuisine)


It is interesting to note that Asian cuisines occupy 4 of the top top types of cuisine (1: Chinese, 3: Indian, 9: Thai, 1: Asian), yet there isn't a single business offering Asian cuisine in the top 10 most commonly found places to eat. This begins to illsutrate the trend that even though business offering Asian cuisine are numerous, they tend to be independent establishments rather than franchises or chains.

### The total reported bicycle parking cpacity

We can't make any assumptions for records where 'capacity' is known.

In [147]:
from bson.code import Code

result = collection.inline_map_reduce(
    Code("function() {"
         "    if (this.capacity) {"
         "        emit('total_bicycle_parking_capacity', Number(this.capacity));"
         "    }"
         "}"), 
    Code("function(key, values) {"
         "    var total = 0;"
         "    for (var i = 0; i < values.length; i++) {"
         "        total += values[i];"
         "    }"
         "    return total;"
         "}"),
    query = {"amenity": "bicycle_parking"});
result

[{u'_id': u'total_bicycle_parking_capacity', u'value': 8756.0}]

### The 5 streets with the most bus stops

For bus stop nodes where the street name is known.

In [137]:
# 
results = collection.aggregate([
        {"$match": {"naptan": {"$exists": 1}, "highway": "bus_stop"}},
        {"$group": {"_id": "$naptan.Street", "num_bus_stops": {"$sum": 1}}},
        {"$project": {"_id": 0, "num_bus_stops": 1, "street_name": "$_id"}},
        {"$sort": {"num_bus_stops": -1}}, 
        {"$limit": 5}
    ])
for result in results:
    print result

{u'street_name': u'Oxford Road', u'num_bus_stops': 43}
{u'street_name': u'Banbury Road', u'num_bus_stops': 40}
{u'street_name': u'Woodstock Road', u'num_bus_stops': 33}
{u'street_name': u'High Street', u'num_bus_stops': 23}
{u'street_name': u'Cowley Road', u'num_bus_stops': 20}


## Conclusion

I am pleasantly surprised to see just how much information about Oxford is encoded within the OSM data. That said, the ineviatble nature of crowd sourced information is clearly present. There is quite a lot of inconsistency in the data (not all illustrated in this project) as well as records that do little more than act as meta information about the creation of the data itself (such as 'fixme' and 'note').

In [159]:
meta_keys = [key for key in unique_key_names if 'fixme' in key.lower() or 'note' in key.lower()]
percentage = '%.2f' % (float(len(meta_keys)) / len(unique_key_names) * 100)
print "There are {} 'meta' tag key names making up {}% of all unique tag key names.".format(len(meta_keys), percentage)
print "\n"
print [key for key in unique_key_names if 'fixme' in key.lower() or 'note' in key.lower()]

There are 78 'meta' tag key names making up 7.72% of all unique tag key names.


['note2', 'Fixme:responce_2', 'frequency:note', 'note:17', 'note:14', 'note:15', 'note:13', 'note:10', 'note:11', 'notes', 'oneway:note', 'building:levels:underground:note', 'continental_geometry:note', 'parking:fixme', 'FIXME:nsl', 'note:highway', 'parking:note', 'fixme:lane', 'Note:2', 'maxspeed:note', 'FIXME', 'note:bicycle', 'note:layer', 'Fixme:2', 'fixme:responce_3', 'fixme:responce_5', 'fixme:responce_4', 'note:alt_name', 'fixme', 'note:designation', 'note:cont', 'note:crossing', 'name:note', 'highway:note', 'naptan:Notes', 'FIXME:responce', 'FIXME:cont', 'note:reply', 'note:name', 'note', 'layer:note', 'note:16', 'note:barrier', 'not:name:note', 'bicycle:note', 'note:geometry', 'note:frequency', 'note:surface', 'alignment_note', 'note:park_and_ride', 'note:0.1', 'note:0.2', 'Note', 'crossing:note', 'payment:notes', 'note:4', 'note:5', 'note:6', 'note:7', 'note:0', 'note:2', 'note:3', 'note:8', 'not

### Resources

The production of this project was waided by information found on the following website (various authors and contributors)

- [Udacity.com](udacity.com)
- [Docs.MongoDB.com](docs.mongodb.com)
- [API.MongoDB.com/python](api.mongodb.com/python)
- [StackOverflow.com](stackoverflow.com)